# Construcción y Uso de Una Curva Cero Cupón

## Configuración

Se importa la versión de `QC_Financial` compilada para Python3.

In [1]:
import qcfinancial as qcf

Librerías adicionales.

In [2]:
import pandas as pd

Para formateo de `pandas.DataFrames`.

In [3]:
format_dict = {
    'nominal': '{0:,.2f}',
    'amort': '{0:,.2f}',
    'interes': '{0:,.2f}',
    'flujo': '{0:,.2f}',
    'amortizacion': '{0:,.2f}',
    'icp_inicial': '{0:,.2f}',
    'icp_final': '{0:,.2f}',
    'uf_inicial': '{0:,.2f}',
    'uf_final': '{0:,.2f}',
    'valor_tasa': '{0:,.4%}',
    'spread': '{0:,.4%}',
    'gearing': '{0:,.2f}',
    'amort_moneda_pago': '{0:,.2f}',
    'interes_moneda_pago': '{0:,.2f}',
    'valor_indice_fx': '{0:,.2f}'
}

## Construcción de la Curva

La construcción de una curva se hace en varios pasos.

### Vectores de `Float` e `Int`

In [4]:
# Este es un vector de números enteros (grandes, de ahí la l (long))
lvec = qcf.long_vec()

In [5]:
# Agregar un elemento
lvec.append(1000)

In [6]:
# Este es un vector de números double.
vec = qcf.double_vec()

In [7]:
# Agregar un elemento
vec.append(.025)

In [8]:
# Obtener ese elemento
print("Tasa: {0:,.2%}".format(vec[0]))

Tasa: 2.50%


### Objeto Curva

Es simplemente un `long_vec` que representa las abscisas de la curva y un `double_vec` que representa las ordenadas. Ambos vectores deben tener el mismo largo. 

In [9]:
zcc = qcf.QCCurve(lvec, vec)

Un elemento de una curva se representa como un par abscisa, ordenada.

In [10]:
zcc.get_values_at(0)

(1000, 0.025)

Se obtiene el plazo en una posición de la curva.

In [11]:
print(zcc.get_values_at(0))

(1000, 0.025)


Se obtiene la tasa en una posición de la curva.

In [12]:
zcc.get_values_at(0)[1]

0.025

Se agrega un par (plazo, valor) a la curva.

In [13]:
zcc.set_pair(100, .026)

Se verifica.

In [14]:
# Plazo
zcc.get_values_at(1)[0]

1000

In [15]:
# Valor
zcc.get_values_at(1)[1]

0.025

Se agrega un par más.

In [16]:
zcc.set_pair(370, .03)

Se itera sobre la curva mostrando sus valores

In [17]:
for i in range(0, zcc.get_length()):
    pair = zcc.get_values_at(i)
    print("Tenor: {0:} Valor: {1:.4%}".format(pair[0], pair[1]))

Tenor: 100 Valor: 2.6000%
Tenor: 370 Valor: 3.0000%
Tenor: 1000 Valor: 2.5000%


Se define un interpolador. En este caso, un interpolador lineal.

In [18]:
lin = qcf.QCLinearInterpolator(zcc)

Se hace una prueba.

In [19]:
plazo = 120
print(f"Tasa a {plazo:.0f} días es igual a {lin.interpolate_at(plazo):.4%}")

Tasa a 120 días es igual a 2.6296%


Para completar el proceso se define una fracción de año, un factor de capitalización y un tipo de tasa. Con estos objetos se termina de dar de alta una curva cero.

In [20]:
yf = qcf.QCAct360()
wf = qcf.QCLinearWf()
tasa = qcf.QCInterestRate(.01, yf, wf)

In [21]:
zz = qcf.ZeroCouponCurve(lin, tasa)

El interpolador permite obtener una tasa a cualquier plazo.

In [22]:
plazo = 365
print("Tasa en {0:} es igual a {1:.4%}".format(plazo, zz.get_rate_at(plazo)))

Tasa en 365 es igual a 2.9926%


In [23]:
type(zz)

qcfinancial.ZeroCouponCurve

In [24]:
zz.get_discount_factor_at(1)

0.9999277829934504

#### Otros métodos:

Tasa Forward

In [25]:
d1 = 30
d2 = 90
print("Tasa forward entre los días {0:} y {1:}: {2:.4%}".format(
    d1, d2, zz.get_forward_rate(d1, d2)))

Tasa forward entre los días 30 y 90: 2.5944%


Derivada del factor de capitalización de la Tasa Forward. El argumento representa el índice de la tasa de la curva.

In [26]:
zz.fwd_wf_derivative_at(0)

0.1659467849041197

## Valorizar

Se da de alta un objeto `PresentValue`.

In [27]:
pv = qcf.PresentValue()

### Depósito a Plazo

Se utilizará como instrumento un depósito a plazo en CLP o USD. Este instrumento se modela como un `SimpleCashflow`. Este, a su vez se construye con un monto, una fecha y una moneda.

In [28]:
# Con estas variables vamos a construir
fecha_vcto = qcf.QCDate(12, 1, 2021)
monto = 10_000_000.0
clp = qcf.QCCLP()

# Se construye el depósito
depo = qcf.SimpleCashflow(fecha_vcto, monto, clp)

In [29]:
print("Monto del depósito: {0:,.0f}".format(depo.amount()))

Monto del depósito: 10,000,000


Se define una fecha de valorización y se calcula el valor presente del depo.

In [30]:
fecha_hoy = qcf.QCDate(17, 1, 2020)
print("Valor presente depo: {0:,.2f}".format(pv.pv(fecha_hoy, depo, zz)))

Valor presente depo: 9,709,212.68


Se verifica *a mano* el resultado.

In [31]:
plazo = fecha_hoy.day_diff(fecha_vcto)
print("Plazo:", plazo)

Plazo: 361


In [32]:
tasa_int = zz.get_rate_at(plazo)
print("Tasa: {0:,.4%}".format(tasa_int))

Tasa: 2.9867%


In [33]:
valor_presente = monto / (1 + tasa_int * plazo / 360)
print("Valor presente a mano: {0:,.2f}".format(valor_presente))

Valor presente a mano: 9,709,212.68


### Renta Fija Local

Se muestra el ejemplo de valorización de un bono bullet a tasa fija con las convenciones de la Bolsa de Comercio. Para el ejemplo usamos las características del BTU0150326.

Se dan de alta los parámetros requeridos para instanciar un objeto de tipo `FixedRateLeg`.

In [34]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(1, 3, 2015)
fecha_final = qcf.QCDate(1, 3, 2026)
bus_adj_rule = qcf.BusyAdjRules.NO
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 100.0
amort_es_flujo = True
valor_tasa_fija = .015
tasa_cupon = qcf.QCInterestRate(
    valor_tasa_fija, 
    qcf.QC30360(),
    qcf.QCLinearWf()
)
moneda = qcf.QCCLP()
es_bono = True

# Se da de alta el objeto
pata_bono = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

Se da de alta el valor de la TERA y luego se construye un objeto de tipo `ChileanFixedRateBond`.

In [35]:
tera = qcf.QCInterestRate(.015044, qcf.QCAct365(), qcf.QCCompoundWf())
bono_chileno = qcf.ChileanFixedRateBond(pata_bono, tera)

Se valoriza al 2021-09-28 a una TIR de mercado del 1.61%.

In [36]:
fecha_valor = qcf.QCDate(28, 9, 2021)
tir = qcf.QCInterestRate(.0161, qcf.QCAct365(), qcf.QCCompoundWf())

valor_presente = bono_chileno.present_value(fecha_valor, tir)
precio = bono_chileno.precio(fecha_valor, tir)
valor_par = bono_chileno.valor_par(fecha_valor)

print(f'Valor presente: {valor_presente:,.8f}')
print(f'Precio: {precio:,.2%}')
print(f'Valor par: {valor_par:,.18f}')

Valor presente: 99.67188455
Precio: 99.56%
Valor par: 100.110516628864033351


Con esto el valor a pagar es:

In [37]:
valor_uf = 30_080.37
valor_pago = precio * valor_par * valor_uf
print(f'Valor a pagar: {valor_pago:,.0f}')

Valor a pagar: 2,998,111


Con 4 decimales en el precio (4 decimales porcentuales, 6 decimales en el número):

In [38]:
precio2 = bono_chileno.precio2(fecha_valor, tir, 6)
print(f'Precio a 4 decmales: {precio2:.4%}')

Precio a 4 decmales: 99.5619%


La función `precio2` entrega el mismo resultado que la función precio cuando se utiliza con 2 decimales porcentuales.

In [39]:
precio22 = bono_chileno.precio2(fecha_valor, tir, 4)
print(f'Precio a 4 decmales: {precio22:.4%}')

Precio a 4 decmales: 99.5600%


Se muestran las diferencias con la convención de precio usual en mercados desarrollados.

In [40]:
bono = qcf.FixedRateBond(pata_bono)
print(f'Valor presente: {bono.present_value(fecha_valor, tir):,.8f}')
print(f'Precio: {bono.price(fecha_valor, tir):,.8f}')

Valor presente: 99.67188455
Precio: 99.55938455


### Curvas Reales

Construyamos dos curvas a partir de data real. Primero la curva CAMARACLP.

In [45]:
curva = pd.read_excel("./input/curva_clp.xlsx")
curva.style.format({"tasa": "{0:,.4%}"})

,curva,fecha,plazo,tasa
0,CAMARACLP,2020-03-05 00:00:00,1,1.7500%
1,CAMARACLP,2020-03-05 00:00:00,4,1.7501%
2,CAMARACLP,2020-03-05 00:00:00,96,1.4867%
3,CAMARACLP,2020-03-05 00:00:00,188,1.3049%
4,CAMARACLP,2020-03-05 00:00:00,279,1.2870%
5,CAMARACLP,2020-03-05 00:00:00,369,1.3002%
6,CAMARACLP,2020-03-05 00:00:00,553,1.3035%
7,CAMARACLP,2020-03-05 00:00:00,734,1.2951%
8,CAMARACLP,2020-03-05 00:00:00,1099,1.4440%
9,CAMARACLP,2020-03-05 00:00:00,1465,1.6736%


Se da de alta un vector con los plazos (variable de tipo `long`) y un vector con las tasas (variable de tipo `double`).

In [46]:
lvec1 = qcf.long_vec()
vec1 = qcf.double_vec()
for index, row in curva.iterrows():
    lvec1.append(int(row['plazo']))
    vec1.append(row['tasa'])

Luego, con una curva, un interpolador y un objeto `QCInterestRate`(que indica la convención de las tasas de la curva) se construye una curva cupón cero.

In [47]:
zcc1 = qcf.QCCurve(lvec1, vec1)
lin1 = qcf.QCLinearInterpolator(zcc1)
zz1 = qcf.ZeroCouponCurve(lin1, tasa)

Luego, la curva LIBORUSD3M.

In [49]:
curva_libor = pd.read_excel("./input/curva_usd.xlsx")
curva_libor.style.format({"tasa": "{0:,.4%}"})

,curva,fecha,plazo,tasa
0,LIBORUSD3MBBG,2020-01-22 00:00:00,3,1.5362%
1,LIBORUSD3MBBG,2020-01-22 00:00:00,4,1.1521%
2,LIBORUSD3MBBG,2020-01-22 00:00:00,7,1.5536%
3,LIBORUSD3MBBG,2020-01-22 00:00:00,14,1.5850%
4,LIBORUSD3MBBG,2020-01-22 00:00:00,31,1.6595%
5,LIBORUSD3MBBG,2020-01-22 00:00:00,60,1.7698%
6,LIBORUSD3MBBG,2020-01-22 00:00:00,91,1.8010%
7,LIBORUSD3MBBG,2020-01-22 00:00:00,123,1.7711%
8,LIBORUSD3MBBG,2020-01-22 00:00:00,152,1.7542%
9,LIBORUSD3MBBG,2020-01-22 00:00:00,182,1.7394%


In [50]:
lvec2 = qcf.long_vec()
vec2 = qcf.double_vec()
for index, row in curva_libor.iterrows():
    lvec2.append(int(row['plazo']))
    vec2.append(row['tasa'])

zcc2 = qcf.QCCurve(lvec2, vec2)
lin2 = qcf.QCLinearInterpolator(zcc2)
zz2 = qcf.ZeroCouponCurve(lin2, tasa)

Finalmente, la curva CAMARACLF.

In [51]:
curva_camara_clf = pd.read_excel("./input/curva_clf.xlsx")
curva_camara_clf.style.format({"tasa": "{0:,.4%}"})

,curva,fecha,plazo,tasa
0,CAMARACLF,2020-03-05 00:00:00,1,-5.6780%
1,CAMARACLF,2020-03-05 00:00:00,4,-5.6744%
2,CAMARACLF,2020-03-05 00:00:00,35,-0.9340%
3,CAMARACLF,2020-03-05 00:00:00,64,-2.1183%
4,CAMARACLF,2020-03-05 00:00:00,96,-2.0079%
5,CAMARACLF,2020-03-05 00:00:00,126,-2.0762%
6,CAMARACLF,2020-03-05 00:00:00,155,-1.9197%
7,CAMARACLF,2020-03-05 00:00:00,188,-1.9347%
8,CAMARACLF,2020-03-05 00:00:00,218,-1.7626%
9,CAMARACLF,2020-03-05 00:00:00,249,-1.7987%


In [52]:
lvec3 = qcf.long_vec()
vec3 = qcf.double_vec()
for index, row in curva_camara_clf.iterrows():
    lvec3.append(int(row['plazo']))
    vec3.append(row['tasa'])

zcc3 = qcf.QCCurve(lvec3, vec3)
lin3 = qcf.QCLinearInterpolator(zcc3)
zz3 = qcf.ZeroCouponCurve(lin3, tasa)

#### Curvas para Sensibilidad

Se define que vértice de la curva se quiere desplazar.

In [53]:
vertice = 13

Se construyen las curvas con ese vértice 1 punto básico más arriba y 1 punto básico más abajo.

In [54]:
bp = .0001
vec_sens_up = qcf.double_vec()
vec_sens_down = qcf.double_vec()
for index, row in curva.iterrows():
    if index == vertice:
        vec_sens_up.append(row['tasa'] + bp)
        vec_sens_down.append(row['tasa'] - bp)
    else:
        vec_sens_up.append(row['tasa'])
        vec_sens_down.append(row['tasa'])

zcc_sens_up = qcf.QCCurve(lvec1, vec_sens_up)
lin_sens_up = qcf.QCLinearInterpolator(zcc_sens_up)
zz_sens_up = qcf.ZeroCouponCurve(lin_sens_up, tasa)

zcc_sens_down = qcf.QCCurve(lvec1, vec_sens_down)
lin_sens_down = qcf.QCLinearInterpolator(zcc_sens_down)
zz_sens_down = qcf.ZeroCouponCurve(lin_sens_down, tasa)

In [55]:
vec2_sens_up = qcf.double_vec()
vec2_sens_down = qcf.double_vec()
for index, row in curva_libor.iterrows():
    if index == vertice:
        vec2_sens_up.append(row['tasa'] + bp)
        vec2_sens_down.append(row['tasa'] - bp)
    else:
        vec2_sens_up.append(row['tasa'])
        vec2_sens_down.append(row['tasa'])

zcc2_sens_up = qcf.QCCurve(lvec2, vec2_sens_up)
lin2_sens_up = qcf.QCLinearInterpolator(zcc2_sens_up)
zz2_sens_up = qcf.ZeroCouponCurve(lin2_sens_up, tasa)

zcc2_sens_down = qcf.QCCurve(lvec2, vec2_sens_down)
lin2_sens_down = qcf.QCLinearInterpolator(zcc2_sens_down)
zz2_sens_down = qcf.ZeroCouponCurve(lin2_sens_down, tasa)

In [56]:
vec3_sens_up = qcf.double_vec()
vec3_sens_down = qcf.double_vec()
for index, row in curva_camara_clf.iterrows():
    if index == vertice:
        vec3_sens_up.append(row['tasa'] + bp)
        vec3_sens_down.append(row['tasa'] - bp)
    else:
        vec3_sens_up.append(row['tasa'])
        vec3_sens_down.append(row['tasa'])

zcc3_sens_up = qcf.QCCurve(lvec3, vec3_sens_up)
lin3_sens_up = qcf.QCLinearInterpolator(zcc3_sens_up)
zz3_sens_up = qcf.ZeroCouponCurve(lin3_sens_up, tasa)

zcc3_sens_down = qcf.QCCurve(lvec3, vec3_sens_down)
lin3_sens_down = qcf.QCLinearInterpolator(zcc3_sens_down)
zz3_sens_down = qcf.ZeroCouponCurve(lin3_sens_down, tasa)

### FixedRateCashflow Leg

Se da de alta una pata fija:

In [57]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(12, 11, 2019)
fecha_final = qcf.QCDate(12, 11, 2020)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.SHORTFRONT
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 20_000_000.0
amort_es_flujo = True
valor_tasa_fija = .01774
tasa_cupon = qcf.QCInterestRate(
    valor_tasa_fija, 
    qcf.QC30360(), 
    qcf.QCLinearWf()
)
moneda = qcf.QCUSD()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

In [58]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_leg.size()):
    tabla.append(qcf.show(fixed_rate_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = list(qcf.get_column_names("FixedRateCashflow", ""))
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-12,2020-05-12,2020-05-12,"20,000,000.00",0.00,"177,400.00",True,"177,400.00",USD,1.7740%,Lin30360
1,2020-05-12,2020-11-12,2020-11-12,"20,000,000.00","20,000,000.00","177,400.00",True,"20,177,400.00",USD,1.7740%,Lin30360


Se calcula ahora el valor presente:

In [59]:
vp_fija = pv.pv(fecha_hoy, fixed_rate_leg, zz2)
print("Valor presente de la pata fija es: {0:,.0f}".format(vp_fija))

Valor presente de la pata fija es: 20,072,981


Al calcular el valor presente, también se calculan las derivadas del valor presente respecto a cada uno de los vértices de la curva.

In [60]:
der = pv.get_derivatives()

Con esas derivadas, se puede calcular la sensibilidad a la curva cupón cero a un movimiento de 1 punto básico.

In [61]:
i = 0
bp = .0001
total = 0
for d in der:
    total += d * bp
    print("Sensibilidad en {0:}: {1:0,.0f}".format(i, d * bp))
    i += 1
print("Sensibilidad total: {0:,.0f}".format(total))

Sensibilidad en 0: 0
Sensibilidad en 1: 0
Sensibilidad en 2: 0
Sensibilidad en 3: 0
Sensibilidad en 4: 0
Sensibilidad en 5: 0
Sensibilidad en 6: -1
Sensibilidad en 7: -4
Sensibilidad en 8: 0
Sensibilidad en 9: 0
Sensibilidad en 10: 0
Sensibilidad en 11: 0
Sensibilidad en 12: -282
Sensibilidad en 13: -1,353
Sensibilidad en 14: 0
Sensibilidad en 15: 0
Sensibilidad en 16: 0
Sensibilidad en 17: 0
Sensibilidad en 18: 0
Sensibilidad en 19: 0
Sensibilidad en 20: 0
Sensibilidad en 21: 0
Sensibilidad en 22: 0
Sensibilidad en 23: 0
Sensibilidad en 24: 0
Sensibilidad en 25: 0
Sensibilidad en 26: 0
Sensibilidad en 27: 0
Sensibilidad total: -1,641


Se puede verificar la sensibilidad por diferencias finitas.

Se calcula el valor presente con las curvas desplazadas.

In [62]:
vp_fija_sens_up = pv.pv(fecha_hoy, fixed_rate_leg, zz2_sens_up)
vp_fija_sens_down = pv.pv(fecha_hoy, fixed_rate_leg, zz2_sens_down)
print("Valor presente up de la pata fija es: {0:,.0f}".format(vp_fija_sens_up))
print("Valor presente down de la pata fija es: {0:,.0f}".format(
    vp_fija_sens_down))

Valor presente up de la pata fija es: 20,071,628
Valor presente down de la pata fija es: 20,074,334


Finalmente, se calcula la sensibilidad (usando la aproximación central por diferencias finitas).

In [63]:
print("Sensibilidad por diferencias finitas: {0:,.0f}".format(
    (vp_fija_sens_up - vp_fija_sens_down) / 2))

Sensibilidad por diferencias finitas: -1,353


Tanto el VP como la sensibilidad coinciden con lo que muestra FD en la pata fija de la operación 2879.

### IcpClpCashflow2 Leg

In [64]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(10,1,2019)
fecha_final = qcf.QCDate(10,7,2029)
bus_adj_rule = qcf.BusyAdjRules.FOLLOW
periodicidad_pago = qcf.Tenor('2Y')
periodo_irregular_pago = qcf.StubPeriod.SHORTFRONT
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 38_000_000_000.0
amort_es_flujo = True
spread = .0
gearing = 1.0

icp_clp2_leg = qcf.LegFactory.build_bullet_icp_clp2_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    spread,
    gearing,
    True
)

In [65]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp2_leg.size()):
    tabla.append(qcf.show(icp_clp2_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = list(qcf.get_column_names("IcpClpCashflow2", ""))
df9 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df9.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-01-10,2019-07-10,2019-07-10,"38,000,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
1,2019-07-10,2021-07-12,2021-07-12,"38,000,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
2,2021-07-12,2023-07-10,2023-07-10,"38,000,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
3,2023-07-10,2025-07-10,2025-07-10,"38,000,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
4,2025-07-10,2027-07-12,2027-07-12,"38,000,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
5,2027-07-12,2029-07-10,2029-07-10,"38,000,000,000.00","38,000,000,000.00",True,"38,000,000,000.00",CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360


Notar que al dar de alta un Leg con IcpClpCashflow2, los valores futuros de los ICP son los default (=10,000.00). Por lo tanto, el primer paso para valorizar estos cashflows, es calcular los valores forward de los índices.

Se comienza dando de alta un objeto de tipo `ForwardRates`.

In [66]:
fwd_rates = qcf.ForwardRates()

Se calculan los índices forward.

In [67]:
icp_val = 18_890.34 # icp a la fecha de proceso
fecha_hoy = qcf.QCDate(5, 3, 2020)

for i in range(icp_clp2_leg.size()):
    cashflow = icp_clp2_leg.get_cashflow_at(i)
    if cashflow.get_start_date() <= fecha_hoy <= cashflow.get_end_date():
            index = i

icp_fecha_inicio_cupon_vigente = 18_376.69
icp_clp2_leg.get_cashflow_at(index).set_start_date_icp(icp_fecha_inicio_cupon_vigente)

fwd_rates.set_rates_icp_clp_leg(fecha_hoy, icp_val, icp_clp2_leg, zz1)

In [68]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp2_leg.size()):
    tabla.append(qcf.show(icp_clp2_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = list(qcf.get_column_names("IcpClpCashflow2", ""))
df9 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df9.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-01-10,2019-07-10,2019-07-10,"38,000,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
1,2019-07-10,2021-07-12,2021-07-12,"38,000,000,000.00",0.00,True,"1,760,277,860.28",CLP,"18,376.69","19,227.96",2.2800%,"1,764,086,666.67",0.0000%,1.00,LinAct360
2,2021-07-12,2023-07-10,2023-07-10,"38,000,000,000.00",0.00,True,"1,260,461,341.38",CLP,"19,227.96","19,865.75",1.6400%,"1,260,248,888.89",0.0000%,1.00,LinAct360
3,2023-07-10,2025-07-10,2025-07-10,"38,000,000,000.00",0.00,True,"2,219,270,452.18",CLP,"19,865.75","21,025.94",2.8800%,"2,222,240,000.00",0.0000%,1.00,LinAct360
4,2025-07-10,2027-07-12,2027-07-12,"38,000,000,000.00",0.00,True,"2,921,878,806.15",CLP,"21,025.94","22,642.66",3.7800%,"2,920,680,000.00",0.0000%,1.00,LinAct360
5,2027-07-12,2029-07-10,2029-07-10,"38,000,000,000.00","38,000,000,000.00",True,"41,014,662,342.55",CLP,"22,642.66","24,438.98",3.9200%,"3,016,440,000.00",0.0000%,1.00,LinAct360


Con esto, podemos calcular el valor presente.

In [69]:
vp_icp_clp = pv.pv(fecha_hoy, icp_clp2_leg, zz1)
print("Valor presente pata ICPCLP: {0:,.0f}".format(vp_icp_clp))

Valor presente pata ICPCLP: 39,062,144,488


In [70]:
csh = icp_clp2_leg.get_cashflow_at(5)

In [71]:
pv.pv(fecha_hoy, csh, zz1)

31702674802.50826

También en este caso es posible calcular la sensibilidad a la curva de descuento.

In [72]:
der = pv.get_derivatives()
i = 0
bp = .0001
for d in der:
    print("Sensibilidad en {0:}: {1:0,.0f}".format(i, d * bp))
    i += 1

Sensibilidad en 0: -0
Sensibilidad en 1: -0
Sensibilidad en 2: -0
Sensibilidad en 3: -0
Sensibilidad en 4: -0
Sensibilidad en 5: -0
Sensibilidad en 6: -0
Sensibilidad en 7: -0
Sensibilidad en 8: -0
Sensibilidad en 9: -0
Sensibilidad en 10: -0
Sensibilidad en 11: -0
Sensibilidad en 12: -0
Sensibilidad en 13: -0
Sensibilidad en 14: -15,407,642
Sensibilidad en 15: -7,831,157
Sensibilidad en 16: -0
Sensibilidad en 17: -0
Sensibilidad en 18: -0


Podemos ver la sensibilidad total:

In [73]:
sens_disc = [d * bp for d in der]
print("Sensibilidad de descuento: {0:,.0f} CLP".format(sum(sens_disc)))

Sensibilidad de descuento: -23,238,799 CLP


La estructura es la misma que para una pata fija, lo que indica que se debe también incluir la sensibilidad a la curva de proyección.

In [74]:
import numpy as np
bp = .0001
result = []
for i in range(icp_clp2_leg.size()):
    cshflw = icp_clp2_leg.get_cashflow_at(i)
    amt_der = cshflw.get_amount_derivatives()
    df = zz1.get_discount_factor_at(fecha_hoy.day_diff(cshflw.get_settlement_date()))
    amt_der = [a * bp * df for a in amt_der]
    if len(amt_der) > 0:
        result.append(np.array(amt_der))
total = result[0] * 0

for r in result:
    total += r

for i in range(len(total)):
    print("Sensibilidad en {0:}: {1:0,.0f}".format(i, total[i]))

print("Sensibilidad de proyección: {0:,.0f} CLP".format(sum(total)))

Sensibilidad en 0: 0
Sensibilidad en 1: 0
Sensibilidad en 2: 0
Sensibilidad en 3: 0
Sensibilidad en 4: 0
Sensibilidad en 5: 74,757
Sensibilidad en 6: 158,384
Sensibilidad en 7: 0
Sensibilidad en 8: 256,858
Sensibilidad en 9: 130,015
Sensibilidad en 10: 644,319
Sensibilidad en 11: 327,485
Sensibilidad en 12: 998,768
Sensibilidad en 13: 518,033
Sensibilidad en 14: 15,407,642
Sensibilidad en 15: 7,831,157
Sensibilidad en 16: 0
Sensibilidad en 17: 0
Sensibilidad en 18: 0
Sensibilidad de proyección: 26,347,418 CLP


Como se espera de una pata ICPCLP (con lag de pago igual a 0 y spread igual a 0), ambas sensibilidades se cancelan.

#### Se verifica la sensibilidad de proyección por diferencias finitas:

In [75]:
fwd_rates.set_rates_icp_clp_leg(fecha_hoy, icp_val, icp_clp2_leg, zz_sens_up)
vp_icp_clp_up = pv.pv(fecha_hoy, icp_clp2_leg, zz1)

fwd_rates.set_rates_icp_clp_leg(fecha_hoy, icp_val, icp_clp2_leg, zz_sens_down)
vp_icp_clp_down = pv.pv(fecha_hoy, icp_clp2_leg, zz1)

print("Sensibilidad en vértice {0:}: {1:,.0f} CLP".format(
    vertice, (vp_icp_clp_up - vp_icp_clp_down) / 2))

Sensibilidad en vértice 13: 518,033 CLP


### IborCashflow2 Leg

In [76]:
### Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(12, 11, 2019)
fecha_final = qcf.QCDate(12, 11, 2029)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
periodicidad_fijacion = qcf.Tenor('3M')
periodo_irregular_fijacion = qcf.StubPeriod.NO

# vamos a usar el mismo calendario para pago y fijaciones
lag_de_fijacion = 2

# Definición del índice
codigo = 'LIBORUSD3M'
lin_act360 = qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf())
fixing_lag = qcf.Tenor('2d')
tenor = qcf.Tenor('3m')
fixing_calendar = calendario
settlement_calendar = calendario
usd = qcf.QCUSD()
libor_usd_3m = qcf.InterestRateIndex(
    codigo,
    lin_act360,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    usd
)
# Fin índice

nominal = 20_000_000.0
amort_es_flujo = True
moneda = usd
spread = .0
gearing = 1.0

# Es la op 2879 en FD
ibor_leg = qcf.LegFactory.build_bullet_ibor2_leg(
    rp, 
    fecha_inicio, 
    fecha_final, 
    bus_adj_rule, 
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    periodicidad_fijacion, 
    periodo_irregular_fijacion,
    calendario, 
    lag_de_fijacion, 
    libor_usd_3m,
    nominal, 
    amort_es_flujo, 
    moneda, 
    spread, 
    gearing
)

In [77]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_leg.size()):
    tabla.append(qcf.show(ibor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha_fixing', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda', 'codigo_indice', 'valor_tasa', 'spread', 'gearing', 'tipo_tasa']
df5 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df5.style.format(format_dict)

,fecha_inicial,fecha__final,fecha_fixing,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,codigo_indice,valor_tasa,spread,gearing,tipo_tasa
0,2019-11-12,2020-02-12,2019-11-08,2020-02-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
1,2020-02-12,2020-05-12,2020-02-10,2020-05-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
2,2020-05-12,2020-08-12,2020-05-08,2020-08-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
3,2020-08-12,2020-11-12,2020-08-10,2020-11-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
4,2020-11-12,2021-02-12,2020-11-10,2021-02-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
5,2021-02-12,2021-05-12,2021-02-10,2021-05-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
6,2021-05-12,2021-08-12,2021-05-10,2021-08-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
7,2021-08-12,2021-11-12,2021-08-10,2021-11-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
8,2021-11-12,2022-02-14,2021-11-10,2022-02-14,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
9,2022-02-14,2022-05-12,2022-02-10,2022-05-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360


In [78]:
libor = 0.0190063
fecha_hoy = qcf.QCDate(25, 2, 2020)
ibor_leg.get_cashflow_at(0).set_rate_value(libor)
fwd_rates.set_rates_ibor_leg(fecha_hoy, ibor_leg, zz2)

In [79]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_leg.size()):
    tabla.append(qcf.show(ibor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha_fixing', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda', 'codigo_indice', 'valor_tasa', 'spread', 'gearing', 'tipo_tasa']
df5 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df5.style.format(format_dict)

,fecha_inicial,fecha__final,fecha_fixing,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,codigo_indice,valor_tasa,spread,gearing,tipo_tasa
0,2019-11-12,2020-02-12,2019-11-08,2020-02-12,"20,000,000.00",0.00,"97,143.31",True,"97,143.31",USD,LIBORUSD3M,1.9006%,0.0000%,1.00,LinAct360
1,2020-02-12,2020-05-12,2020-02-10,2020-05-12,"20,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
2,2020-05-12,2020-08-12,2020-05-08,2020-08-12,"20,000,000.00",0.00,"87,136.92",True,"87,136.92",USD,LIBORUSD3M,1.7049%,0.0000%,1.00,LinAct360
3,2020-08-12,2020-11-12,2020-08-10,2020-11-12,"20,000,000.00",0.00,"83,357.57",True,"83,357.57",USD,LIBORUSD3M,1.6309%,0.0000%,1.00,LinAct360
4,2020-11-12,2021-02-12,2020-11-10,2021-02-12,"20,000,000.00",0.00,"79,834.14",True,"79,834.14",USD,LIBORUSD3M,1.5620%,0.0000%,1.00,LinAct360
5,2021-02-12,2021-05-12,2021-02-10,2021-05-12,"20,000,000.00",0.00,"75,388.80",True,"75,388.80",USD,LIBORUSD3M,1.5247%,0.0000%,1.00,LinAct360
6,2021-05-12,2021-08-12,2021-05-10,2021-08-12,"20,000,000.00",0.00,"75,147.35",True,"75,147.35",USD,LIBORUSD3M,1.4703%,0.0000%,1.00,LinAct360
7,2021-08-12,2021-11-12,2021-08-10,2021-11-12,"20,000,000.00",0.00,"75,919.01",True,"75,919.01",USD,LIBORUSD3M,1.4854%,0.0000%,1.00,LinAct360
8,2021-11-12,2022-02-14,2021-11-10,2022-02-14,"20,000,000.00",0.00,"76,541.33",True,"76,541.33",USD,LIBORUSD3M,1.4657%,0.0000%,1.00,LinAct360
9,2022-02-14,2022-05-12,2022-02-10,2022-05-12,"20,000,000.00",0.00,"72,657.16",True,"72,657.16",USD,LIBORUSD3M,1.5033%,0.0000%,1.00,LinAct360


In [80]:
which_cashflow = 1
d1 = fecha_hoy.day_diff(ibor_leg.get_cashflow_at(which_cashflow).get_start_date())
d2 = fecha_hoy.day_diff(ibor_leg.get_cashflow_at(which_cashflow).get_end_date())
print("d1: {0:,.0f}".format(d1))
print("d2: {0:,.0f}".format(d2))
crv = zz2
w1 = 1 / crv.get_discount_factor_at(d1)
w2 = 1 / crv.get_discount_factor_at(d2)
print("Factor forward: {0:.4%}".format(w2 / w1))
print("Tasa forward: {0:.4%}".format((w2 / w1 - 1) * 360 / (d2 - d1)))
print("Curve method {0:.4%}".format(crv.get_forward_rate_with_rate(libor_usd_3m.get_rate(), d1, d2)))

d1: -13
d2: 77
Factor forward: 100.4379%
Tasa forward: 1.7517%
Curve method 1.5288%


In [81]:
vp_ibor = pv.pv(fecha_hoy, ibor_leg, zz2)
print("Valor presente pata IBOR: {0:,.0f}".format(vp_ibor))

Valor presente pata IBOR: 19,923,921


In [82]:
der = pv.get_derivatives()
i = 0
bp = .0001
for d in der:
    print("Sensibilidad en {0:}: {1:0,.0f}".format(i, d * bp))
    i += 1
print("Sensibilidad de descuento: {0:,.0f} USD".format(sum(der) * bp))

Sensibilidad en 0: 0
Sensibilidad en 1: 0
Sensibilidad en 2: 0
Sensibilidad en 3: 0
Sensibilidad en 4: 0
Sensibilidad en 5: 0
Sensibilidad en 6: 0
Sensibilidad en 7: 0
Sensibilidad en 8: -2
Sensibilidad en 9: -2
Sensibilidad en 10: 0
Sensibilidad en 11: -3
Sensibilidad en 12: -3
Sensibilidad en 13: 0
Sensibilidad en 14: -3
Sensibilidad en 15: -10
Sensibilidad en 16: -21
Sensibilidad en 17: -47
Sensibilidad en 18: -85
Sensibilidad en 19: -114
Sensibilidad en 20: -234
Sensibilidad en 21: -1,895
Sensibilidad en 22: -13,122
Sensibilidad en 23: 0
Sensibilidad en 24: 0
Sensibilidad en 25: 0
Sensibilidad en 26: 0
Sensibilidad en 27: 0
Sensibilidad de descuento: -15,541 USD


#### Se verifica la sensibilidad de descuento por diferencias finitas.

In [83]:
vp_ibor_up = pv.pv(fecha_hoy, ibor_leg, zz2_sens_up)
print("Valor presente up pata IBOR: {0:,.0f}".format(vp_ibor_up))

vp_ibor_down = pv.pv(fecha_hoy, ibor_leg, zz2_sens_down)
print("Valor presente down pata IBOR: {0:,.0f}".format(vp_ibor_down))

print("Sensibilidad de descuento en el vértice {0:}: {1:,.0f}".format(vertice, (vp_ibor_up - vp_ibor_down) / 2))

Valor presente up pata IBOR: 19,923,921
Valor presente down pata IBOR: 19,923,921
Sensibilidad de descuento en el vértice 13: 0


Se calcula también la sensibilidad a la curva de proyección.

In [84]:
import numpy as np
bp = .0001
result = []

for i in range(ibor_leg.size()):
    cshflw = ibor_leg.get_cashflow_at(i)
    df = zz2.get_discount_factor_at(fecha_hoy.day_diff(cshflw.get_settlement_date()))
    amt_der = cshflw.get_amount_derivatives()
    if len(amt_der) > 0:
        amt_der = [a * bp * df for a in amt_der]
        result.append(np.array(amt_der))

total = result[0] * 0
for r in result:
    total += r

for i in range(len(total)):
    print("Sensibilidad en {0:}: {1:0,.0f}".format(i, total[i]))

print("Sensibilidad de proyección: {0:,.0f} USD".format(sum(total)))

Sensibilidad en 0: 0
Sensibilidad en 1: 0
Sensibilidad en 2: 0
Sensibilidad en 3: 0
Sensibilidad en 4: 0
Sensibilidad en 5: -192
Sensibilidad en 6: -233
Sensibilidad en 7: 0
Sensibilidad en 8: 2
Sensibilidad en 9: 2
Sensibilidad en 10: 0
Sensibilidad en 11: 3
Sensibilidad en 12: 3
Sensibilidad en 13: 0
Sensibilidad en 14: 3
Sensibilidad en 15: 10
Sensibilidad en 16: 21
Sensibilidad en 17: 1
Sensibilidad en 18: 81
Sensibilidad en 19: 193
Sensibilidad en 20: 258
Sensibilidad en 21: 1,820
Sensibilidad en 22: 13,222
Sensibilidad en 23: 0
Sensibilidad en 24: 0
Sensibilidad en 25: 0
Sensibilidad en 26: 0
Sensibilidad en 27: 0
Sensibilidad de proyección: 15,194 USD


#### Se verifica la sensibilidad de proyección por diferencias finitas.

In [85]:
fwd_rates.set_rates_ibor_leg(fecha_hoy, ibor_leg, zz2_sens_up)
vp_ibor_up = pv.pv(fecha_hoy, ibor_leg, zz2)
print("Valor presente up pata IBOR: {0:,.0f}".format(vp_ibor_up))

fwd_rates.set_rates_ibor_leg(fecha_hoy, ibor_leg, zz2_sens_down)
vp_ibor_down = pv.pv(fecha_hoy, ibor_leg, zz2)
print("Valor presente down pata IBOR: {0:,.0f}".format(vp_ibor_down))

print("Sensibilidad de proyección en el vértice {0:}: {1:,.0f}".format(vertice, (vp_ibor_up - vp_ibor_down) / 2))

Valor presente up pata IBOR: 19,923,921
Valor presente down pata IBOR: 19,923,921
Sensibilidad de proyección en el vértice 13: 0


### IcpClfCashflow Leg

In [86]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 5, 2018)
fecha_final = qcf.QCDate(9, 5, 2026) 
bus_adj_rule = qcf.BusyAdjRules.FOLLOW
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.SHORTFRONT
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 6000_00.0
amort_es_flujo = True 
spread = .0
gearing = 1.0

icp_clf_leg = qcf.LegFactory.build_bullet_icp_clf_leg(
    rp, 
    fecha_inicio, 
    fecha_final, 
    bus_adj_rule, 
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    nominal, 
    amort_es_flujo, 
    spread, 
    gearing
)

In [87]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clf_leg.size()):
    tabla.append(qcf.show(icp_clf_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'uf_inicial', 'uf_final',
            'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df8 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df8.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,uf_inicial,uf_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2018-05-31,2018-11-09,2018-11-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
1,2018-11-09,2019-05-09,2019-05-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
2,2019-05-09,2019-11-11,2019-11-11,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
3,2019-11-11,2020-05-11,2020-05-11,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
4,2020-05-11,2020-11-09,2020-11-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
5,2020-11-09,2021-05-10,2021-05-10,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
6,2021-05-10,2021-11-09,2021-11-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
7,2021-11-09,2022-05-09,2022-05-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
8,2022-05-09,2022-11-09,2022-11-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
9,2022-11-09,2023-05-09,2023-05-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360


In [88]:
icp_hoy = 18_882.07
uf_hoy = 28440.19
fwd_rates.set_rates_icp_clf_leg(fecha_hoy, icp_hoy, uf_hoy, icp_clf_leg, zz1, zz1, zz3)
cshflw = icp_clf_leg.get_cashflow_at(3)
cshflw.set_start_date_uf(28080.26)
cshflw.set_start_date_icp(18786.13)

In [89]:
print(fecha_hoy)

2020-02-25


In [90]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clf_leg.size()):
    tabla.append(qcf.show(icp_clf_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'uf_inicial', 'uf_final',
            'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df8 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df8.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,uf_inicial,uf_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2018-05-31,2018-11-09,2018-11-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
1,2018-11-09,2019-05-09,2019-05-09,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
2,2019-05-09,2019-11-11,2019-11-11,"600,000.00",0.00,True,0.00,CLF,"10,000.00","10,000.00","35,000.00","35,000.00",0.0000%,0.00,0.0000%,1.00,LinAct360
3,2019-11-11,2020-05-11,2020-05-11,"600,000.00",0.00,True,"-7,178.78",CLF,"18,786.13","18,943.62","28,080.26","28,658.55",-2.3649%,"-7,173.53",0.0000%,1.00,LinAct360
4,2020-05-11,2020-11-09,2020-11-09,"600,000.00",0.00,True,"-5,300.72",CLF,"18,943.62","19,056.79","28,658.55","29,086.73",-1.7492%,"-5,305.91",0.0000%,1.00,LinAct360
5,2020-11-09,2021-05-10,2021-05-10,"600,000.00",0.00,True,"-3,909.59",CLF,"19,056.79","19,182.42","29,086.73","29,470.52",-1.2929%,"-3,921.80",0.0000%,1.00,LinAct360
6,2021-05-10,2021-11-09,2021-11-09,"600,000.00",0.00,True,"-4,842.13",CLF,"19,182.42","19,306.95","29,470.52","29,903.16",-1.5847%,"-4,833.34",0.0000%,1.00,LinAct360
7,2021-11-09,2022-05-09,2022-05-09,"600,000.00",0.00,True,"-5,431.38",CLF,"19,306.95","19,440.27","29,903.16","30,384.70",-1.8038%,"-5,441.46",0.0000%,1.00,LinAct360
8,2022-05-09,2022-11-09,2022-11-09,"600,000.00",0.00,True,"-3,700.83",CLF,"19,440.27","19,606.92","30,384.70","30,835.35",-1.2040%,"-3,692.27",0.0000%,1.00,LinAct360
9,2022-11-09,2023-05-09,2023-05-09,"600,000.00",0.00,True,"-2,958.63",CLF,"19,606.92","19,794.39","30,835.35","31,284.45",-0.9825%,"-2,963.88",0.0000%,1.00,LinAct360


In [91]:
vp_icp_clf = pv.pv(fecha_hoy, icp_clf_leg, zz3)
print("Valor presente en UF: {0:,.2f}".format(vp_icp_clf))
print("Valor presente en CLP: {0:,.0f}".format(vp_icp_clf * uf_hoy))

Valor presente en UF: 595,431.99
Valor presente en CLP: 16,934,198,846


In [92]:
print("Dif %: {0:.4%}".format(12_940.56/12_943.45-1))

Dif %: -0.0223%


In [93]:
der = pv.get_derivatives()
i = 0
bp = .0001
for d in der:
    print("Sensibilidad en {0:}: {1:0,.2f}".format(i, d * bp))
    i += 1
print("Sensibilidad de descuento: {0:,.2f} CLF".format(sum(der) * bp))

Sensibilidad en 0: 0.00
Sensibilidad en 1: 0.00
Sensibilidad en 2: 0.00
Sensibilidad en 3: 0.10
Sensibilidad en 4: 0.06
Sensibilidad en 5: 0.00
Sensibilidad en 6: 0.00
Sensibilidad en 7: 0.00
Sensibilidad en 8: 0.00
Sensibilidad en 9: 0.27
Sensibilidad en 10: 0.12
Sensibilidad en 11: 0.00
Sensibilidad en 12: 0.00
Sensibilidad en 13: 0.00
Sensibilidad en 14: 0.00
Sensibilidad en 15: 0.32
Sensibilidad en 16: 0.18
Sensibilidad en 17: 0.00
Sensibilidad en 18: 0.00
Sensibilidad en 19: 0.00
Sensibilidad en 20: 0.65
Sensibilidad en 21: 1.63
Sensibilidad en 22: 2.15
Sensibilidad en 23: 1.29
Sensibilidad en 24: -0.26
Sensibilidad en 25: -335.81
Sensibilidad en 26: -82.41
Sensibilidad en 27: 0.00
Sensibilidad en 28: 0.00
Sensibilidad en 29: 0.00
Sensibilidad en 30: 0.00
Sensibilidad en 31: 0.00
Sensibilidad en 32: 0.00
Sensibilidad de descuento: -411.73 CLF


In [94]:
bp = .0001
result = []

for i in range(icp_clf_leg.size()):
    cshflw = icp_clf_leg.get_cashflow_at(i)
    df = zz3.get_discount_factor_at(fecha_hoy.day_diff(cshflw.date()))
    amt_der = cshflw.get_amount_ufclf_derivatives()
    if len(amt_der) > 0:
        amt_der = [a * bp * df for a in amt_der]
        result.append(np.array(amt_der))

total = result[0] * 0
for r in result:
    total += r

for i in range(len(total)):
    print("Sensibilidad en {0:}: {1:0,.2f}".format(i, total[i]))

Sensibilidad en 0: 0.00
Sensibilidad en 1: 0.00
Sensibilidad en 2: 0.00
Sensibilidad en 3: -0.10
Sensibilidad en 4: -0.06
Sensibilidad en 5: 0.00
Sensibilidad en 6: 0.00
Sensibilidad en 7: 0.00
Sensibilidad en 8: 0.00
Sensibilidad en 9: -0.27
Sensibilidad en 10: -0.12
Sensibilidad en 11: 0.00
Sensibilidad en 12: 0.00
Sensibilidad en 13: 0.00
Sensibilidad en 14: 0.00
Sensibilidad en 15: -0.32
Sensibilidad en 16: -0.18
Sensibilidad en 17: 0.00
Sensibilidad en 18: 0.00
Sensibilidad en 19: 0.00
Sensibilidad en 20: -0.65
Sensibilidad en 21: -1.63
Sensibilidad en 22: -2.15
Sensibilidad en 23: -1.29
Sensibilidad en 24: 0.26
Sensibilidad en 25: 335.81
Sensibilidad en 26: 82.41
Sensibilidad en 27: 0.00
Sensibilidad en 28: 0.00
Sensibilidad en 29: 0.00
Sensibilidad en 30: 0.00
Sensibilidad en 31: 0.00
Sensibilidad en 32: 0.00


### CompoundedOvernightRate Leg

In [95]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(3, 1, 2022)
fecha_final = qcf.QCDate(3, 1, 2023)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0

######################################################################
# Definición del índice

codigo = 'OISTEST'
lin_act360 = qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf())
fixing_lag = qcf.Tenor('0d')
tenor = qcf.Tenor('1d')
fixing_calendar = calendario
settlement_calendar = calendario
usd = qcf.QCUSD()
oistest = qcf.InterestRateIndex(
    codigo,
    lin_act360,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    usd)

# Fin índice
######################################################################

nominal = 1000000.0
amort_es_flujo = True
moneda = usd
spread = .01
gearing = 1.0

cor_leg = qcf.LegFactory.build_bullet_compounded_overnight_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago,
    calendario,
    lag_pago,
    calendario,
    oistest,
    nominal,
    amort_es_flujo,
    usd,
    spread,
    gearing,
    True,
    8,
    0,
    0
)

In [96]:
ts = qcf.time_series()
ts[qcf.QCDate(27, 12, 2021)] = .01
ts[qcf.QCDate(28, 12, 2021)] = .02
ts[qcf.QCDate(29, 12, 2021)] = .03
ts[qcf.QCDate(30, 12, 2021)] = .04
ts[qcf.QCDate(31, 12, 2021)] = .04

#### Valor Presente

In [97]:
fwd_rates.set_rates_compounded_overnight_leg(
    fecha_inicio,
    cor_leg,
    zz1,
    ts
)

In [98]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, cor_leg.size()):
    tabla.append(qcf.show(cor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda', 'indice', 'valor_tasa', 'spread', 'gearing', 'tipo_tasa']
df9 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df9.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,indice,valor_tasa,spread,gearing,tipo_tasa
0,2022-01-03,2022-04-04,2022-04-04,"1,000,000.00",0.00,"2,527.78",True,"6,322.09",USD,OISTEST,1.5010%,1.0000%,1.00,LinAct360
1,2022-04-04,2022-07-04,2022-07-04,"1,000,000.00",0.00,"2,527.78",True,"5,379.73",USD,OISTEST,1.1282%,1.0000%,1.00,LinAct360
2,2022-07-04,2022-10-03,2022-10-03,"1,000,000.00",0.00,"2,527.78",True,"5,618.72",USD,OISTEST,1.2228%,1.0000%,1.00,LinAct360
3,2022-10-03,2023-01-03,2023-01-03,"1,000,000.00","1,000,000.00","2,555.56",True,"1,005,930.44",USD,OISTEST,1.3206%,1.0000%,1.00,LinAct360


In [99]:
print(f'{pv.pv(fecha_inicio, cor_leg, zz1):,.0f}')

1,010,055


In [100]:
pres_value = 0.0
for i in range(0, cor_leg.size()):
    cshflw = cor_leg.get_cashflow_at(i)
    print(f'{cshflw.amount():,.2f}, {cshflw.get_amortization()}')
    temp = pv.pv(fecha_inicio, cshflw, zz1)
    # print(f'{temp:,.0f}')
    pres_value += temp
print(f'{pres_value:,.0f}')

6,322.09, 0.0
5,379.73, 0.0
5,618.72, 0.0
1,005,930.44, 1000000.0
1,010,055


#### Sensibilidad

In [101]:
pips = .0001 # 1 punto básico

##### Curva de Proyección

Estas derivadas se deben obtener inmediatamente después de calcular el valor presente.

In [102]:
proj_sens_by_cashflow = np.array([np.array(
    np.array(cor_leg.get_cashflow_at(i).get_amount_derivatives()) *
    zz1.get_discount_factor_at(fecha_inicio.day_diff(cor_leg.get_cashflow_at(i).get_settlement_date())) * pips)
                             for i in range(cor_leg.size())])
proj_sens = np.sum(proj_sens_by_cashflow, axis=0)

##### Curva de Descuento

In [103]:
disc_der = np.array(pv.get_derivatives()) * pips